In [1]:
import pandas as pd
r = pd.read_csv( 'ratings.csv' )

In [2]:
r

,user_id,book_id,rating
0,1,258,5
1,2,4081,4
2,2,260,5
3,2,9296,5
4,2,2318,3
...,...,...,...
5976474,49925,510,5
5976475,49925,528,4
5976476,49925,722,4
5976477,49925,949,5


In [14]:
len(r['book_id'].unique())

10000

In [22]:
reviews_per_user = r.groupby( 'user_id' ).user_id.apply( lambda x: len( x ))
reviews_per_user.describe([.25, .5, .75, .95])

count    53424.000000
mean       111.868804
std         26.071224
min         19.000000
25%         96.000000
50%        111.000000
75%        128.000000
95%        157.000000
max        200.000000
Name: user_id, dtype: float64

In [109]:
reviews_per_user.quantile(0.95)

157.0

Step-1) Extract users and ratings of more than 157
extract the user ids who have given more than 157 ratings and when we will have user ids we will extract the ratings of only this user id from the rating dataframe

In [2]:
xu = r['user_id'].value_counts() > 157
yu = xu[xu].index  #user_ids
print(yu.shape)
uratings = r[r['user_id'].isin(yu)]
uratings

(2534,)


,user_id,book_id,rating
977,75,3254,2
978,75,6777,5
979,75,11,5
981,75,372,4
982,75,476,3
...,...,...,...
5976327,20586,1089,3
5976328,20586,894,5
5976329,20586,755,3
5976330,20586,2109,3


In [24]:
reviews_per_book = r.groupby( 'book_id' ).book_id.apply( lambda x: len( x ))
reviews_per_book.describe([.25, .5, .75, .95])

count    10000.000000
mean       597.647900
std       1267.289788
min          8.000000
25%        155.000000
50%        248.000000
75%        503.000000
95%       2132.350000
max      22806.000000
Name: book_id, dtype: float64

In [44]:
reviews_per_book

book_id
1        22806
2        21850
3        16931
4        19088
5        16604
         ...  
9996       141
9997        93
9998       102
9999       130
10000      111
Name: book_id, Length: 10000, dtype: int64

step-2) Extract books that have received more than 2132 ratings.
Now dataframe size has decreased and we have 4.8 lakh because when we merge the dataframe, all the book id-data we were not having. Now we will count the rating of each book so we will group data based on title and aggregate based on rating.

In [3]:
xb = r['book_id'].value_counts() > 2132
yb = xb[xb].index  #user_ids
print(yb.shape)
bratings = r[r['book_id'].isin(yb)]
bratings

(500,)


,user_id,book_id,rating
0,1,258,5
5,2,26,4
7,2,33,4
8,2,301,5
12,4,70,4
...,...,...,...
5976456,3173,459,5
5976458,53358,253,5
5976466,35336,317,5
5976473,49925,330,5


In [43]:
reviews_per_book_filtered =bratings.groupby( 'book_id' ).book_id.apply( lambda x: len( x ))
reviews_per_book_filtered

book_id
1      22806
2      21850
3      16931
4      19088
5      16604
       ...  
715     2400
730     2147
733     2533
773     2225
826     2175
Name: book_id, Length: 500, dtype: int64

In [4]:
final_rating=pd.merge(uratings, bratings, on=['user_id', 'book_id', 'rating'])

In [29]:
final_rating

,user_id,book_id,rating
0,75,11,5
1,75,372,4
2,75,476,3
3,75,115,4
4,75,122,4
...,...,...,...
200942,25030,178,3
200943,50898,209,3
200944,39663,74,4
200945,19951,267,5


In [22]:
len(final_rating['user_id'].unique())

2534

replace book_id by title in final_rating

b = pd.read_csv('books.csv' )

In [5]:
b2=b[['book_id', 'title']]

NameError: name 'b' is not defined

In [61]:
frt=final_rating.merge(b2, on='book_id')

In [105]:
frt

,user_id,book_id,rating,title
0,75,11,5,The Kite Runner
1,143,11,5,The Kite Runner
2,230,11,5,The Kite Runner
3,247,11,5,The Kite Runner
4,256,11,4,The Kite Runner
...,...,...,...,...
200942,27330,279,5,Harry Potter and the Cursed Child - Parts One ...
200943,20782,279,1,Harry Potter and the Cursed Child - Parts One ...
200944,51250,279,2,Harry Potter and the Cursed Child - Parts One ...
200945,50898,279,3,Harry Potter and the Cursed Child - Parts One ...


In [24]:
book_pivot = final_rating.pivot_table(columns='user_id', index='book_id', values="rating")
book_pivot.fillna(0, inplace=True)

In [23]:
book_pivot.describe()

user_id,35,75,143,173,178,202,215,230,247,256,...,53145,53165,53173,53245,53279,53281,53292,53293,53318,53366
count,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.00000,500.000000,500.000000,500.000000,...,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000
mean,0.604000,0.630000,0.570000,0.836000,0.422000,0.824000,0.57400,0.680000,0.594000,0.676000,...,0.638000,0.734000,0.428000,0.582000,0.514000,0.852000,1.270000,0.880000,0.520000,0.424000
std,1.300217,1.411767,1.407502,1.590649,1.238946,1.658471,1.40872,1.536986,1.478356,1.465373,...,1.578588,1.663966,1.252179,1.388211,1.228341,1.638128,2.178668,1.906008,1.399112,1.167454
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,0.000000,0.000000,0.000000
max,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.00000,5.000000,5.000000,5.000000,...,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000


In [26]:
book_pivot

user_id,35,75,143,173,178,202,215,230,247,256,...,53145,53165,53173,53245,53279,53281,53292,53293,53318,53366
book_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,4.0,5.0,4.0,4.0,0.0,0.0,5.0,0.0,3.0,...,5.0,5.0,0.0,4.0,4.0,0.0,0.0,0.0,4.0,4.0
2,0.0,5.0,5.0,5.0,0.0,4.0,0.0,0.0,4.0,3.0,...,5.0,5.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,4.0
3,0.0,0.0,3.0,4.0,1.0,4.0,0.0,0.0,0.0,3.0,...,5.0,4.0,0.0,0.0,1.0,3.0,5.0,0.0,2.0,5.0
4,0.0,0.0,0.0,4.0,0.0,4.0,0.0,5.0,4.0,4.0,...,5.0,0.0,0.0,5.0,0.0,0.0,5.0,0.0,0.0,3.0
5,0.0,0.0,0.0,0.0,0.0,4.0,3.0,4.0,5.0,4.0,...,5.0,5.0,0.0,0.0,0.0,4.0,5.0,0.0,0.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
715,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,4.0,0.0,...,0.0,0.0,0.0,0.0,0.0,3.0,0.0,5.0,0.0,0.0
730,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,5.0,0.0,0.0
733,4.0,0.0,0.0,0.0,0.0,3.0,0.0,5.0,3.0,0.0,...,0.0,0.0,0.0,4.0,0.0,0.0,0.0,5.0,0.0,0.0


In [28]:
book_pivot.loc[826]

user_id
35       0.0
75       0.0
143      0.0
173      0.0
178      0.0
        ... 
53281    0.0
53292    0.0
53293    5.0
53318    0.0
53366    0.0
Name: 826, Length: 2534, dtype: float64

In [17]:
final_rating['user_id'].unique()

array([   75,   143,   173, ..., 34531, 51553,  7553])

In [19]:
53366 in final_rating['user_id'].unique()

True

In [20]:
final_rating['book_id'].unique()

array([ 11, 372, 476, 115, 122, 109,  22, 150,  63, 378,  18, 516,  38,
        92,  23,  94,  45,  35,  55,  10,  27,  21,  24,   2,  26,   8,
       426, 100, 301, 325, 457,  57, 258, 255, 265,  14, 531, 646, 410,
       114, 132, 162, 383, 138, 130,  90, 101, 119,   7,   9, 170, 354,
        70, 127, 650,  15,  32,  60, 213, 595,   4, 730, 129,   5,  13,
       306, 284,  65, 405, 715, 401, 264, 148, 456, 297, 217, 212,  33,
       225, 131,  89,  82, 538, 181, 468, 282, 304,  43, 704, 272, 557,
       495, 733, 238, 400, 198, 233, 597, 160, 561, 121, 173, 637,  48,
        72, 211, 117, 230,  76, 424,  64, 186, 270,  75, 570, 189, 350,
       155, 102, 126,  19,  95, 289, 118, 219,  28,  71, 125, 436, 205,
       696, 371, 407, 461, 375, 288,  80, 546, 103,  54, 492,  42, 496,
       180, 530, 510, 330,  87, 413, 620, 113,  93,  44, 136, 487, 660,
       575, 693, 108, 287,  78,  40, 111, 161, 335, 373, 158,  97, 245,
        58, 278,  50,  37, 339, 133, 257, 386,  29, 171, 326, 54

In [29]:
type(final_rating['book_id'].unique())

numpy.ndarray

In [64]:
from scipy.sparse import csr_matrix
book_sparse = csr_matrix(book_pivot)

In [65]:
from sklearn.neighbors import NearestNeighbors
model = NearestNeighbors(n_neighbors=10,algorithm='brute')
model.fit(book_sparse)

NearestNeighbors(algorithm='brute', n_neighbors=10)

In [67]:
distances, suggestions = model.kneighbors(book_pivot.iloc[0, :].values.reshape(1, -1))

In [68]:
for i in range(len(suggestions)):
  print(book_pivot.index[suggestions[i]])

Index([''Salem's Lot', 'The Dead Zone', 'Cujo', 'Christine', 'Pet Sematary',
       'Needful Things', 'Firestarter', 'Carrie', 'Under the Dome', 'Misery'],
      dtype='object', name='title')


# SVD with surprise library
predict a rating given by an user

In [8]:
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate
reader = Reader()
data = Dataset.load_from_df(final_rating[['user_id', 'book_id', 'rating']], reader)

In [10]:
svd = SVD()

In [ ]:
cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

We get a mean Root Mean Sqaure Error of 0.87 approx which is more than good enough for our case. Let us now train on our dataset and arrive at predictions.

In [11]:
trainset = data.build_full_trainset()
svd.fit(trainset)

In [97]:
from surprise import accuracy
from surprise.model_selection import KFold

testset = trainset.build_testset()
predictions = svd.test(testset)
# RMSE should be low as we are biased
accuracy.rmse(predictions, verbose=True)  # ~ 0.64 (which is low)

RMSE: 0.6372


0.6372250291284687

In [100]:
from surprise import NormalPredictor
np=NormalPredictor()
np.fit(trainset)
testset = trainset.build_testset()
predictions = np.test(testset)
# RMSE should be low as we are biased
accuracy.rmse(predictions, verbose=True) 

RMSE: 1.3739


1.373856595484506

In [104]:
from surprise import NMF
nmf=NMF()
nmf.fit(trainset)
testset = trainset.build_testset()
predictions = np.test(testset)
# RMSE should be low as we are biased
accuracy.rmse(predictions, verbose=True) 

RMSE: 1.3760


1.3759858946144512

In [12]:
#pick user with user Id 75 and check the ratings she/he has given.
final_rating[final_rating['user_id'] == 53318]

,user_id,book_id,rating
190292,53318,188,3
190293,53318,182,4
190294,53318,441,5
190295,53318,97,1
190296,53318,239,4
...,...,...,...
190352,53318,670,5
190353,53318,675,5
190354,53318,696,3
190355,53318,488,5


In [103]:
svd.predict(53318, 188)
# à débugger, pour un même utilisateur, peu importe l'item, l'estimation est la même

Prediction(uid=53318, iid=188, r_ui=None, est=3.9929947406674553, details={'was_impossible': False})

For book with ID 112, we get an estimated prediction of 3.425. One startling feature of this recommender system is that it doesn't care what the movie is (or what it contains). It works purely on the basis of an assigned movie ID and tries to predict ratings based on how the other users have predicted the movie.

# Sources

https://www.kaggle.com/ibtesama/getting-started-with-a-movie-recommendation-system#Content-Based-Filtering

https://www.analyticsvidhya.com/blog/2021/06/build-book-recommendation-system-unsupervised-learning-project/

https://towardsdatascience.com/how-you-can-build-simple-recommender-systems-with-surprise-b0d32a8e4802

https://github.com/AmolMavuduru/SurprisePythonExamples

https://towardsdatascience.com/item-based-collaborative-filtering-in-python-91f747200fab

https://github.com/shamafarabi/Book_Recommendation_Engine

https://www.kdnuggets.com/2020/07/building-content-based-book-recommendation-engine.html

https://towardsdatascience.com/what-should-i-read-next-68be827960bf

https://www.kaggle.com/omarzaghlol/goodreads-2-book-recommender-system

https://chubek.github.io/recommender/2020/12/15/cold-start-problem-copy.html

https://towardsdatascience.com/how-to-build-a-simple-movie-recommender-system-with-tags-b9ab5cb3b616

https://github.com/JohnsonKuan/movie-rec-tags

https://surprise.readthedocs.io/en/stable/matrix_factorization.html#surprise.prediction_algorithms.matrix_factorization.SVD

https://github.com/93tilinfinity/svd-recommender

https://towardsdatascience.com/svd-where-model-tuning-goes-wrong-61c269402919

https://www.oprahdaily.com/entertainment/books/a29576863/types-of-book-genres/

https://github.com/KevinLiao159/MyDataSciencePortfolio/blob/master/movie_recommender/src/als_recommender.py

https://github.com/KevinLiao159/MyDataSciencePortfolio/blob/master/movie_recommender/movie_recommendation_using_NeuMF.ipynb

https://towardsdatascience.com/introduction-to-recommender-systems-1-971bd274f421

https://www.analyticssteps.com/blogs/what-content-based-recommendation-system-machine-learning


# Collaborative filtering
Données utilisées : ratings

Préprocessing : 5% 

. des livres les plus notés

. des utilisateurs les plus collaboratifs (qui ont donné le plus de notes)

Table pivot qui va servir au modèle :

. CF IB

. CF UB

In [ ]:
import pandas as pd
r = pd.read_csv( 'ratings.csv' )
fr = merge_books_users_filter()
rp = pivot_ratings()
def filter_top_5quantile_books():
    reviews_per_book = r.groupby( 'book_id' ).book_id.apply( lambda x: len( x ))
    xb = r['book_id'].value_counts() > reviews_per_book.quantile(0.95)
    yb = xb[xb].index
    bratings = r[r['book_id'].isin(yb)]    
    return bratings
def filter_top_5quantile_users():
    reviews_per_user = r.groupby( 'user_id' ).user_id.apply( lambda x: len( x ))
    xu = r['user_id'].value_counts() > reviews_per_user.quantile(0.95)
    yu = xu[xu].index  #user_ids
    print(yu.shape)
    uratings = r[r['user_id'].isin(yu)]
    return uratings
def merge_books_users_filter():
    uratings = filter_top_5quantile_users()
    bratings = filter_top_5quantile_books()
    return pd.merge(uratings, bratings, on=['user_id', 'book_id', 'rating'])
def pivot_ratings():
    ratings_pivot = fr.pivot_table(columns='user_id', index='book_id', values="rating")
    ratings_pivot.fillna(0, inplace=True)
    return ratings_pivot

## Collabotative filtering item-based

### objectif : 
Proposer un livre similaire à un livre qui a été consulté basé sur la notation des utilisateurs

### Etapes

1 Créer une matrice creuse pour entraîner un modèle knn

2 Proposer les k livres les plus proches à un livre récemment consulté/lu

In [112]:
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
def recommend_similary_books(book_id):
    book_sparse = csr_matrix(rp)
    model = NearestNeighbors(n_neighbors=10,algorithm='brute')
    model.fit(book_sparse)
    distances, suggestions = model.kneighbors(rp.iloc[book_id, :].values.reshape(1, -1))
    for i in range(len(suggestions)):
        print(rp.index[suggestions[i]])

## Collabotative filtering user-based

### objectif : 
Proposer un livre qui est susceptible d'être bien noté par l'utilisateur (au delà de 4.0 par exemple)

### Etapes 
1 Réduire les données d'entraînement : garder 5% 

. des livres qui sont les plus notés

. des utilisateurs qui ont donné le plus de notes

2 Entraîner un modèle svd sur le dataframe pour estimer la note donnée à un livre par un utilisateur (conditions : livre et utilisateur dans les 5% retenus pour le dataframe d'entraînement)

3 Boucler sur toute la liste des livres et proposer les livres avec les meilleures estimations de note 

In [116]:
books_to_rate = book_pivot[book_pivot.loc[:, 53318]==0]

TypeError: '(slice(None, None, None), 53318)' is an invalid key

In [13]:
def build_svd():
    svd=SVD()    
    reader = Reader()
    data = Dataset.load_from_df(final_rating[['user_id', 'book_id', 'rating']], reader)
    trainset = data.build_full_trainset()
    svd.fit(trainset)
    return svd

svd = build_svd()

# step 2 CF UB
def estimate_rating(uid, iid):
    return svd.predict(uid, iid).est

# step 3 CF UB
def recommend_best_est_books(userid, threeshold = 4): 
    # liste des livres que l'utilisateur n'a pas notés
    books_to_rate = book_pivot[book_pivot.loc[:, userid]==0].loc[:, userid]   
    # un dictionnaire pour stocker la note à estimer pour chaque livre
    rating_est={}
    books_to_recommend = []
    # pour chaque livre à noter
    for key, value in books_to_rate.items():
        # ajouter la note estimée au dictionnaire
        rating_est[key]=estimate_rating(userid, key)
        if rating_est[key] >= threeshold :
            books_to_recommend.append(key)
    print (rating_est, books_to_recommend)
    #print ("nombre de livres à recommander : ", len(books_to_recommend))
    #print ("nombre de livres non notés : ", len(rating_est))
    
recommend_best_est_books(53318)

{2: 4.536320652608849, 4: 4.566308875346086, 5: 3.6506668530636457, 7: 4.00191924703036, 9: 3.5966822461546, 10: 4.433079120725114, 11: 4.367642722400431, 12: 4.240534055224804, 15: 4.534014851491184, 16: 4.066287859756868, 19: 4.525886707203237, 22: 3.9552991426748036, 23: 4.610052901050074, 26: 3.876704939670569, 28: 3.906697363752327, 29: 3.878460945838864, 30: 3.7873966495967273, 31: 4.517238113922776, 32: 4.157509107901354, 33: 4.129923055941996, 34: 2.721014549230702, 35: 3.7475801013482175, 36: 4.121294738918336, 38: 3.823754337883272, 39: 3.922087229301363, 40: 3.3433969857779258, 41: 3.7349396896782943, 42: 4.0641280204404175, 43: 4.285517306764438, 44: 3.692294778257733, 45: 3.8937387591600707, 46: 3.9355367205679483, 47: 4.1554082117239455, 48: 3.980655715836488, 49: 2.7561949616310017, 50: 4.562318354772473, 53: 3.369226551394989, 54: 4.243930942692064, 55: 3.4522369025389366, 57: 4.159672551808827, 58: 4.375388470913795, 59: 4.435974094983598, 60: 3.8329249048370726, 61: 4

In [ ]:

# step 3 CF UB
def estimate_books_not_read(userid): 
    # liste des livres que l'utilisateur n'a pas notés
    books_to_rate = book_pivot[book_pivot.loc[:, userid]==0].loc[:, userid]   
    # un dictionnaire pour stocker la note à estimer pour chaque livre
    rating_est={}
    # pour chaque livre à noter
    for key, value in books_to_rate.items():
        # ajouter la note estimée au dictionnaire
        rating_est[key]=estimate_rating(userid, key)
    return (rating_est)
    
def filter_best_est_books(userid, threeshold = 4):
    est = estimate_books_not_read(userid)
    
recommend_best_est_books(53318)

In [140]:
# step 3 CF UB
def recommend_best_est_books(userid, threeshold = 4): 
    # liste des livres que l'utilisateur n'a pas notés
    books_to_rate = book_pivot[book_pivot.loc[:, userid]==0].loc[:, userid]   
    # un dictionnaire pour stocker la note à estimer pour chaque livre
    rating_est={}
    books_to_recommend = []
    # pour chaque livre à noter
    for key, value in books_to_rate.items():
        # ajouter la note estimée au dictionnaire
        rating_est[key]=estimate_rating(userid, key)
        if rating_est[key] >= threeshold :
            books_to_recommend.append(key)
    print (rating_est, books_to_recommend)
    print ("nombre de livres à recommander : ", len(books_to_recommend))
    print ("nombre de livres non notés : ", len(rating_est))
    
recommend_best_est_books(53318)

{'11/22/63': 4.248941658829858, '2001: A Space Odyssey (Space Odyssey, #1)': 4.050467769955871, 'A Child Called "It" (Dave Pelzer #1)': 3.8812127087412565, 'A Christmas Carol': 4.086098926937996, 'A Clash of Kings  (A Song of Ice and Fire, #2)': 4.0116026967509, 'A Clockwork Orange': 3.7785658851609094, 'A Confederacy of Dunces': 3.5372558410663353, 'A Discovery of Witches (All Souls Trilogy, #1)': 3.866251090776611, 'A Farewell to Arms': 3.9503166111002312, 'A Game of Thrones (A Song of Ice and Fire, #1)': 4.36141145109948, 'A Heartbreaking Work of Staggering Genius': 3.9863211873518094, 'A Light in the Attic': 4.390094933460714, 'A Little Princess': 4.487779376154226, 'A Man Called Ove': 4.46175414946937, "A Midsummer Night's Dream": 4.145846387647004, 'A Million Little Pieces': 3.2568504928100785, 'A Prayer for Owen Meany': 4.245824264447997, 'A Separate Peace': 4.142397090558376, 'A Short History of Nearly Everything': 4.239179303813817, 'A Streetcar Named Desire': 4.15803664699394

In [130]:
books_to_rate = book_pivot[book_pivot.loc[:, 53318]==0].loc[:, 53318]

In [131]:
books_to_rate

title
11/22/63                                                             0.0
2001: A Space Odyssey (Space Odyssey, #1)                            0.0
A Child Called "It" (Dave Pelzer #1)                                 0.0
A Christmas Carol                                                    0.0
A Clash of Kings  (A Song of Ice and Fire, #2)                       0.0
                                                                    ... 
Winnie-the-Pooh (Winnie-the-Pooh, #1)                                0.0
Wonder                                                               0.0
Wuthering Heights                                                    0.0
Yes Please                                                           0.0
Zen and the Art of Motorcycle Maintenance: An Inquiry Into Values    0.0
Name: 53318, Length: 435, dtype: float64

In [133]:
rating_est={}
for key, value in books_to_rate.items():
    # ajouter la note estimée au dictionnaire
    rating_est[key]=estimate_rating(53318, key)
print (rating_est)

{'11/22/63': 4.248941658829858, '2001: A Space Odyssey (Space Odyssey, #1)': 4.050467769955871, 'A Child Called "It" (Dave Pelzer #1)': 3.8812127087412565, 'A Christmas Carol': 4.086098926937996, 'A Clash of Kings  (A Song of Ice and Fire, #2)': 4.0116026967509, 'A Clockwork Orange': 3.7785658851609094, 'A Confederacy of Dunces': 3.5372558410663353, 'A Discovery of Witches (All Souls Trilogy, #1)': 3.866251090776611, 'A Farewell to Arms': 3.9503166111002312, 'A Game of Thrones (A Song of Ice and Fire, #1)': 4.36141145109948, 'A Heartbreaking Work of Staggering Genius': 3.9863211873518094, 'A Light in the Attic': 4.390094933460714, 'A Little Princess': 4.487779376154226, 'A Man Called Ove': 4.46175414946937, "A Midsummer Night's Dream": 4.145846387647004, 'A Million Little Pieces': 3.2568504928100785, 'A Prayer for Owen Meany': 4.245824264447997, 'A Separate Peace': 4.142397090558376, 'A Short History of Nearly Everything': 4.239179303813817, 'A Streetcar Named Desire': 4.15803664699394

In [ ]:
df_re=pd.DataFrame.from_dict(rating_est, columns=['book_id', 'rating_est'])
    df_re.set_index('book_id')
    # afficher les livres avec une estimation supérieure au seuil donné
    df_re[df_re['rating_est'] >= threeshold].merge(b2, on = 'book_id')
    

In [ ]:
import difflib
import random

def get_book_id(book_title, metadata):
    
    """
    Gets the book ID for a book title based on the closest match in the metadata dataframe.
    """
    
    existing_titles = list(metadata['title'].values)
    closest_titles = difflib.get_close_matches(book_title, existing_titles)
    book_id = metadata[metadata['title'] == closest_titles[0]]['id'].values[0]
    return book_id

def get_book_info(book_id, metadata):
    
    """
    Returns some basic information about a book given the book id and the metadata dataframe.
    """
    
    book_info = metadata[metadata['id'] == book_id][['id', 'isbn', 
                                                    'authors', 'title', 'original_title']]
    return book_info.to_dict(orient='records')

def predict_review(user_id, book_title, model, metadata):
    
    """
    Predicts the review (on a scale of 1-5) that a user would assign to a specific book. 
    """
    
    book_id = get_book_id(book_title, metadata)
    review_prediction = model.predict(uid=user_id, iid=book_id)
    return review_prediction.est

def generate_recommendation(user_id, model, metadata, thresh=4):
    
    """
    Generates a book recommendation for a user based on a rating threshold. Only
    books with a predicted rating at or above the threshold will be recommended
    """
    
    book_titles = list(metadata['title'].values)
    random.shuffle(book_titles)
    
    for book_title in book_titles:
        rating = predict_review(user_id, book_title, model, metadata)
        if rating >= thresh:
            book_id = get_book_id(book_title, metadata)
            return get_book_info(book_id, metadata)
